In [4]:
from netCDF4 import Dataset, num2date
#we could otherwise import the whole netCDF4 instead that specific functions. 
#if so, to call specific func we type ex netCDF4.num2datea

In [5]:
#now we use the package to open the file and extract the data from the file
#for visualisation is better to use another package
#our file has 4 variables: latitude, longitude, time, temperature(air) 
#for each time, lat and long...it has the value of temperature
import pandas as pd

In [6]:
#our dataset has 4 variables so it s a bit more messy to work with: the time! 
#so it is not a table but a cube
#now we open the file using netCDF4 and extract data
file = 'air.mon.mean.nc'
nc = Dataset(file, mode='r')

In [7]:
nc.variables.keys()

dict_keys(['lat', 'lon', 'time', 'air'])

In [8]:
#we are now storing the values of each variable under the names (lat, lon, air, time)
#so: going to nc then to variables get the key lat and store all values into a variable we call lat
#the last one is a bit funky because its not a number nor a string but is a format called datetime.
#so we want to make sure that python understands we are talking about a date and not just a number (num2date)
# ':' means we want aaall the rows and not just one
lat = nc.variables['lat'][:]
lon = nc.variables['lon'][:]
air = nc.variables['air'][:]
time_var = nc.variables['time']
dtime = num2date(time_var[:], time_var.units)

In [9]:
#since we have 3 dimensions we have to tell pandas that he is excepting a multiIndex
#the first variable names is just a list with the names of the things
#the second line we create a MultiIndex on time, lon and lat
#then we pass the values of air temp
names = ['time', 'lat','lon']
index = pd.MultiIndex.from_product([dtime, lat, lon], names=names)
df = pd.DataFrame ({'Air':air.flatten()}, index=index)

In [10]:
df

Air
time       lat   lon             
1948-01-01  90.0 0.0   -34.926773
                 2.5   -34.926773
                 5.0   -34.926773
                 7.5   -34.926773
                 10.0  -34.926773
...                           ...
2020-01-01 -90.0 347.5 -24.799204
                 350.0 -24.799204
                 352.5 -24.799204
                 355.0 -24.799204
                 357.5 -24.799204

[9092880 rows x 1 columns]

In [11]:
#what this is doing is we are getting rid of that pesky date time values 
#and creating a new column just with the year and another just with the month
year = [row.year for row in dtime]
month =[row.month for row in dtime]

In [12]:
#we are creating the pandas dataframe (the excell-like thing) but now with our new column year
#NB. here we have the option of adding a column for 'month' but my laptop goes KO so I didnt
names = ['year', 'lat', 'lon']
index = pd.MultiIndex.from_product([year, lat, lon], names=names)
df = pd.DataFrame({'Air': air.flatten()}, index=index).reset_index()

In [13]:
df

,year,lat,lon,Air
0,1948,90.0,0.0,-34.926773
1,1948,90.0,2.5,-34.926773
2,1948,90.0,5.0,-34.926773
3,1948,90.0,7.5,-34.926773
4,1948,90.0,10.0,-34.926773
...,...,...,...,...
9092875,2020,-90.0,347.5,-24.799204
9092876,2020,-90.0,350.0,-24.799204
9092877,2020,-90.0,352.5,-24.799204
9092878,2020,-90.0,355.0,-24.799204


In [14]:
df.groupby('year').get_group(2000)

,year,lat,lon,Air
6559488,2000,90.0,0.0,-29.164190
6559489,2000,90.0,2.5,-29.164190
6559490,2000,90.0,5.0,-29.164190
6559491,2000,90.0,7.5,-29.164190
6559492,2000,90.0,10.0,-29.164190
...,...,...,...,...
6685627,2000,-90.0,347.5,-23.232578
6685628,2000,-90.0,350.0,-23.232578
6685629,2000,-90.0,352.5,-23.232578
6685630,2000,-90.0,355.0,-23.232578


In [ ]:
# df.groupby(['lat','lon']).mean()
# df.groupby(['lon','lat','year']).mean()

# the idea is that what the user chooses goes inside the get_group. for example, the user wants to see year==2000
# then the script should run:
# df.groupby('year').get_group(year_chosen_by_user)